## Credit Rating Estimation from Fundamental Data to Estimate Corporate Lending Rate

This code was developed to estimate an interest rate to charge a potential corporate borrower based on a simple risk-based pricing model. The lending rate was calculated to take into account the borrower's credit risks (estimated from financial ratios), which was then used to estimate the Probability of Default (PD). The final loan price was then calculated by adding PD- and Recovery Rate (RR)-based risk premia on top of the current risk-free rate (Altman, 2018). Briefly, the outline of the calculations is as follows:
1. Retrive fundamental accounting data for a listed construction company using `yahooquery` library (Guthrie)
2. Estimate credit rating based on Altman Z-score (Altman, 2005)
3. Based on the estimated credit rating, calculate PD using Standard & Poor’s Average One-Year Transition Rates matrix (Donnelly, 2021)
4. Assign RR based on a current review (Global Credit Data, 2022), as well as other parameters, & calculate final loan rate (Altman, 2018)

In [1]:
# import pandas, numpy & matplotlib for data analysis & visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

# import yahooquery to retrieve accounting data
# and yfinance to retrieve bond yield data
from yahooquery import Ticker
import yfinance as yf

### Section 1 - Retrieve Accounting Data to Calculate Financial Ratios & Altman Z-scores

The following financial ratios were first calculated from the company's financial statements:
- ${X_1}$ = working capital / total assets 
- ${X_2}$ = retained earnings / total assets
- ${X_3}$ = operating income / total assets
- ${X_4}$ = book value of equity / total liabilities

In [2]:
# define ticker symbol of a construction company for which accounting data needs to be retrieved
ticker = 'HOV'

# fetch relevant accounting data using yahooquery & add to dataframe
data = Ticker(ticker)
df = data.balance_sheet()[['asOfDate','TotalAssets','TotalDebt','WorkingCapital','RetainedEarnings']]
df_1 = data.income_statement()[['asOfDate','OperatingIncome']]

# merge data from balance_sheet & income_statement
df = pd.merge(df,df_1,on='asOfDate')

# calculate Book Value of Equity = Total Assets - Total Debt
df['BookValueEquity'] = df['TotalAssets'] - df['TotalDebt']

# set the date as index & drop rows with missing rows
df.set_index('asOfDate',inplace=True)
df.dropna(axis=0, inplace=True)

# show the dataframe
df

,TotalAssets,TotalDebt,WorkingCapital,RetainedEarnings,OperatingIncome,BookValueEquity
asOfDate,,,,,,
2020-10-31,1.827342e+09,1.566232e+09,1.118431e+09,-1.175045e+09,129302000.0,2.611100e+08
2021-10-31,2.320508e+09,1.373462e+09,1.085758e+09,-5.672280e+08,262476000.0,9.470460e+08
2022-10-31,2.562030e+09,1.291352e+09,1.410931e+09,-3.524130e+08,344858000.0,1.270678e+09


The Z-score was then calculated as follows using a modified model as discussed in Altman (2005):
$$
Z-score = 6.56{X_1} + 3.26{X_2} + 6.72{X_3} + 1.05{X_4} + 3.25
$$

In [3]:
# define a function to calculate Altman Z-score

def altman_z_score(data,index):

    X1 = data['WorkingCapital'].iloc[index]/data['TotalAssets'].iloc[index]
    X2 = data['RetainedEarnings'].iloc[index]/data['TotalAssets'].iloc[index]
    X3 = data['OperatingIncome'].iloc[index]/data['TotalAssets'].iloc[index]
    X4 = data['BookValueEquity'].iloc[index]/data['TotalDebt'].iloc[index]

    z_score = (6.56*X1) + (3.26*X2) + (6.72*X3) + (1.05*X4) + 3.25
    
    return z_score

In [4]:
# initiate an empty list
z_list = []

# calculate Altman Z-score for every period
for i in range(len(df)):
    z = altman_z_score(df,i)
    z_list.append(z)

# merge Z-scores with original dataframe
df['Altman Z-score'] = z_list
df['Altman Z-score']

asOfDate
2020-10-31    5.819329
2021-10-31    7.006641
2022-10-31    8.351951
Name: Altman Z-score, dtype: float64

### Section 2 - Estimate Credit Rating Based on Z-scores

In [5]:
# list credit rating categories
rating_list = ['AAA','AA+','AA','AA-','A+','A','A-',
               'BBB+','BBB','BBB-','BB+','BB','BB-','B+','B','B-',
               'CCC+','CCC','CCC-','D']

rating_reversed = list(reversed(rating_list))

# define the cut-off Z-score values for each rating category (Altman, 2005)
bin = [0,1.75,2.5,3.2,3.75,4.15,4.5,4.75,4.95,5.25,5.65,5.85,
        6.25,6.4,6.65,6.85,7,7.3,7.6,8.15,np.inf]

# Assign an estimate of credit rating based on Z-scores (Altman, 2005)
df['Credit Rating'] = pd.cut(x=df['Altman Z-score'],bins=bin,labels=rating_reversed)
df[['Altman Z-score','Credit Rating']]

,Altman Z-score,Credit Rating
asOfDate,,
2020-10-31,5.819329,BBB-
2021-10-31,7.006641,AA-
2022-10-31,8.351951,AAA


### Section 3 - Probability of Default (PD) Estimation

In [6]:
# load S&P Average One-Year Transition Rates matrix (Donnelly, 2021)
matrix = pd.read_csv("transition_matrix.csv").dropna(axis=0)
matrix.reset_index(inplace=True)
matrix.drop(['index','Unnamed: 0','NR'],axis=1, inplace=True)
matrix

,From/to,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,CCC,D
0,AAA,87.03,5.89,2.51,0.69,0.16,0.24,0.13,0.00,0.05,0.00,0.03,0.05,0.03,0.00,0.03,0.00,0.05,0.00
1,AA+,2.31,78.94,10.91,3.54,0.71,0.33,0.19,0.05,0.09,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,AA,0.42,1.31,80.76,8.53,2.72,1.15,0.36,0.39,0.13,0.08,0.05,0.03,0.02,0.02,0.00,0.02,0.05,0.02
3,AA-,0.04,0.11,3.77,78.80,9.68,2.19,0.60,0.25,0.15,0.07,0.03,0.00,0.00,0.03,0.08,0.00,0.00,0.03
4,A+,0.00,0.06,0.44,4.44,78.38,8.73,2.15,0.61,0.34,0.09,0.06,0.09,0.01,0.07,0.03,0.00,0.00,0.05
5,A,0.03,0.04,0.22,0.41,5.32,78.88,6.74,2.38,0.86,0.27,0.10,0.10,0.06,0.08,0.02,0.00,0.01,0.05
6,A-,0.04,0.01,0.06,0.15,0.42,6.49,78.12,7.23,1.98,0.57,0.13,0.13,0.11,0.10,0.02,0.01,0.03,0.06
7,BBB+,0.00,0.01,0.05,0.06,0.20,0.74,7.13,75.83,7.98,1.56,0.36,0.29,0.13,0.15,0.10,0.02,0.06,0.10
8,BBB,0.01,0.01,0.04,0.03,0.10,0.31,1.00,7.73,76.00,6.11,1.34,0.58,0.27,0.22,0.11,0.03,0.05,0.16
9,BBB-,0.01,0.01,0.02,0.04,0.06,0.14,0.25,1.17,9.31,72.40,5.47,2.08,0.83,0.36,0.22,0.16,0.21,0.25


Using the transition matrix, the probability of a particular rating transitioning to D (default) during the next year can be estimated (Donnelly, 2021).

In [7]:
# this code block was modified from Donnelly (2021) to calculate PD

sp_rating_list = ['AAA','AA+','AA','AA-','A+','A','A-',
                  'BBB+','BBB','BBB-','BB+','BB','BB-',
                  'B+','B','B-']

ccc_list = ['CCC+','CCC','CCC-','CC+','CC','CC-','C+','C','C-']

# define a function to estimate PD based on the transition matrix

def prob_default(row):
    
    sp_rating = None

    for i in sp_rating_list:
        if df['Credit Rating'].iloc[row] == i:
            sp_rating = df['Credit Rating'].iloc[row]

    if sp_rating == None:
        for i in ccc_list:
            if df['Credit Rating'].iloc[row] == i:
                sp_rating = 'CCC/C'

    sp_transition_dp = 0

    for i in range(len(matrix)):
        if matrix.iloc[i,0] == sp_rating:
            sp_transition_dp += float(matrix.iloc[i,18])

    return sp_transition_dp

In [8]:
# initiate an empty list
pd_list = []

# calculate PD for every period
for i in range(len(df)):
    p = prob_default(i)
    pd_list.append(p)

# merge Z-scores with original dataframe
df['Probability of Default'] = pd_list
df[['Altman Z-score','Credit Rating','Probability of Default']]

,Altman Z-score,Credit Rating,Probability of Default
asOfDate,,,
2020-10-31,5.819329,BBB-,0.25
2021-10-31,7.006641,AA-,0.03
2022-10-31,8.351951,AAA,0.00


We have attempted to calculate probability of default (PD) from estimated credit ratings derived solely from financial ratios. We can also examine how the credit rating and PD have changed (or not) over the evaluation period. Using the data, we can now calculate the final loan rate to charge the potential borrower.

### Section 4 - Calculations of Final Loan Rate
To calculate the final loan rate, the recovery rate (RR) was assumed to be 75% as published in a study by globalcreditdata.org for the construction industry (Global Credit Data, 2022). Assuming the loan is for a year, the final rate is calculated by including a regulatory capital requirement of 8% as determined by the Bank of International Settlement (BIS) (Bank of International Settlement, 2006) as follows:

$$
Loan\, Rate = Risk\, Free\, Rate\, + \\
        (PD \times (1 - RR)) + \\
        (BIS\, Capital\, Requirement \times Cost\, of\, Equity\, Capital) + \\
        Overhead\, and\, Operation\, Risk\, Charge
$$

The cost of equity capital was assumed to be 15%, and, lastly, the overhead & operation risk charge was assumed to be 0.4% per year (Altman, 2018). The risk-free rate was estimated from the current 10-year Treasury Note yield.

In [9]:
# define a function to calculate loan rate
def loan_rate(riskFreeRate,probOfDefault,
              recoveryRate = 0.75,
              BIS_requires = 0.08,
              costOfEquity = 0.15,
              overheadCharge = 0.004):
    
    loanRate = riskFreeRate + (probOfDefault * (1-recoveryRate)) + (BIS_requires * costOfEquity) + overheadCharge
    
    return loanRate

In [10]:
# risk-free rate is deduced from the latest 10-year T-Note yield
riskFreeRate = yf.download('^TNX',dt.datetime.today()-dt.timedelta(3),dt.datetime.today())['Adj Close']
riskFreeRate = riskFreeRate.iloc[-1]/100 # take the last yield data & convert into decimals
    
# the row index no (r) can be changed to calculate what the loan rate might have been
for r in range(len(df)):
    probOfDefault = df['Probability of Default'].iloc[r]
    loanRate = loan_rate(riskFreeRate,probOfDefault)
    print(f'Loan rate for {ticker} with a credit rating of {df.iloc[r,-2]} as of {str(df.index[r])[:10]} is {round((100*loanRate),3)}%')

[*********************100%***********************]  1 of 1 completed
Loan rate for HOV with a credit rating of BBB- as of 2020-10-31 is 12.373%
Loan rate for HOV with a credit rating of AA- as of 2021-10-31 is 6.873%
Loan rate for HOV with a credit rating of AAA as of 2022-10-31 is 6.123%


The loan rate for this particular company has been calculated based on its risk profile. However, it should be noted that the risk profile was only estimated quantitatively from the company's financial statements. In credit modeling, qualitative aspects often play a critical role in modulating credit rating. Therefore, although the code can be used to arrive at an initial estimate of the loan rate, further investigations are required to adjust the rate according to the nature of the projects, the current economic cycle, or other factors.

### References
- Guthrie, Doug, “yahooquery - Python wrapper for an unofficial Yahoo Finance API”, https://yahooquery.dpguthrie.com/
- Altman, Edward I., “Applications of Distress Prediction Models: What Have We Learned After 50 Years from the Z-Score Models?” *International Journal of Financial Studies*, vol 6, 2018, pp. 70; doi: [10.3390/ijfs6030070](https://www.mdpi.com/2227-7072/6/3/70)
- Altman, Edward I., “An emerging market credit scoring system.” *Emerging Markets Review*, vol 6, 2005, pp. 311-323, https://pages.stern.nyu.edu/~ealtman/emerging_markets_review.pdf
- Global Credit Data, "Observed Recovery Rates Dashboard Corporates." *globalcreditdata.org*, May 2022, https://globalcreditdata.org/wp-content/uploads/2022/05/GCD-Corporates-RR-Dashboard-2022.pdf
- Donnelly, Hugh, "Calculating a Company's Probability of Default with Python." *medium.com*, 4 Mar 2021, https://medium.com/analytics-vidhya/calculating-a-companys-probability-of-default-with-python-5a91cda82caf
- Bank of International Settlements, "Basel II: International Convergence of Capital Measurement and Standards: A Revised Framework - Comprehensive Version." *bis.org*, 2006, https://www.bis.org/publ/bcbs128.pdf